# **- Long run example -**
# **Shortest Path (Dijkstra's) algorithm applied in OSM driving network of the Helsinki Region**

![map1](img/output-map1.png)

## **1. Introduction**
In this example, we are going to process the *Shortest Path* between every buildings registered in OpenStreetMap (OSM) to the Rautatieasema. This practice gives an overview about the *accessibility* of the population to the Helsinki city center considering the central railway station as the central point. The *Dijkstra's algorithm* can be applied in the OSM driving network using the Python library OSMNX which operates network analysis under the hood using NetworkX. The newest version of OSMNX implements *Core Parallelization* so we can operates the routing in parallel in your available cores.

## **2. Datasets**
Both datasets are fetched during the workflow from OSM using the OSMNX and they are:

- Buildings footprint in the Helsinki Region or Uusimaa Administrative Unit (if you want to escalate the process)
- Central railway station - Rautatieasema

## **3. Output**
The process gives as output a layer with 19 070 routes from every buildings in the Helsinki Region to the Rautatieasema

*Find the a map of the results at the beginning of this notebook*

## **4. Objective**
To compare the advantage of processin time of *Shortest Path* algorithm in the Helsinki Region using parallel computing in a 16-core machine

## **5. Resources**

CSC Machine-Puhti:

- *Partition*: small
- *CPU Cores*: 16
- *Memory (GB)*: 60
- *Local Disk (GB)*: 120

## **6. Shortest Path using with 16 Cores**
Follow the instructions and run every cell in the supercomputer.

### **6.1 Import Python libraries**

Be sure that you have installed the OSMNX>=1.6.0 in your environment. Get familiar with this library reading a bit the [OSMNX Documentation](https://osmnx.readthedocs.io/en/stable/user-reference.html)


In [ ]:
import os
import glob
# os.environ['USE_PYGEOS'] = '0'

# for (geospatial) analysis
import geopandas as gpd
from shapely.geometry import MultiLineString
from shapely import ops
import osmnx as ox

import pandas as pd

import matplotlib.pyplot as plt

# for optimization of computational resources
from joblib import Memory
import multiprocessing as mp

# time processing
import time
import warnings
warnings.filterwarnings('ignore')

We are going to create a folder `output` where the results are going to be stored

In [ ]:
# results folder
if not os.path.exists('output'):
    os.makedirs('output')

### **6.2 Fetch OSM residential buildings**

To fetch OSM features you need to define an area like the Helsinki Region and the feature tag like buldings. You can find each one in the next links:

- For OSM areas: [*Nominatim*](https://nominatim.openstreetmap.org/ui/search.html?q=%27Capital+region%2C+Mainland+Finland%2C+Finland%27)
- For feature tags: [*OSM Wiki*](https://wiki.openstreetmap.org/wiki/Buildings)

To fetch the elements from the wanter area we use the osmnx function `features_from_place()`

In [ ]:
# ------> Fetch residential buildings

place_name = 'Capital region, Mainland Finland, Finland'
# place_name = 'Uusimaa, Southern Finland, Mainland Finland, Finland' ---> if you want to escalate 

# Tags for residential buildings
tags = {'building': 'residential'}

# run
all_buildings = ox.features_from_place(place_name, tags)

Once you have fetched the buildings you will see that it contains double index and many columns. We will simplify this to have a more kneat frame of the data simple by deleting the indexes and keeping only the `street name` and the `geometry`.

In [ ]:
# ------> Reset index and get street name

all_buildings = all_buildings.reset_index(drop=False)
all_buildings = all_buildings[['osmid', 'addr:street', 'geometry']]

# rename
all_buildings = all_buildings.rename(columns={'addr:street':'street_name'})

The geometry of the buildings are represented by **points** and **polygons**. What we need to have is only points because the *Shortest Path* analysis runs between two points (longitude and latitude) so we will add the `centroid` of every geometry to remove the polygons and keep them as centroids.

In [ ]:
# fix geometries
all_buildings['geometry'] = [geom.centroid for geom in all_buildings['geometry']]

Here is a quick view of the buildings data.

In [ ]:
all_buildings.head()

The library `geopandas` for spatial analysis contains a function `explore()` that allows you to visualize data in a quick and interactive way. We will add some keywords to modify a bit our visualization

In [ ]:
print(f'Total residential buildings: {len(all_buildings)}')
all_buildings.explore(color='red', marker_kwds={'radius':0.4})

### **6.3 Define Origins as a List**

A particularity of the *Shortest Path* function in OSMNX is that you can process A-Origin to A-Destination. Also, you can process pair routes as Lists in a way that `[A-Origin, B-Origin, C-Origin, ...]` to `[A-Destination, B-Destination, C-Destination, ...]`.

Based on this feature of the OSMNX library we are going to start organizing a List `all_origins` that contains *geometry* objects of every building we have fetched from OSM in the Helsinki Region.

In [ ]:
# --------> define origin List

# list
all_origins = []

for row in all_buildings.itertuples(index=True):
        
    # tuple, only geometries
    geom = row.geometry
    
    all_origins.append(geom)

In [ ]:
all_origins[:10]

### **6.4 Fetching road network**

In this step we are going to fetch the *Graph* object containing the edges (roads) and nodes (interections) of the OSM driving network. The function that we will use is the `graph_from_place()` from OSMNX.

#### *Note!*

We are going to create a function that packs the OSMNX function. The objective is to add it to `cache` memory.

The library `joblib` is able to cache processed that can be repetitive and optimize the processing in a workflow. This means, that the first time you fetch the *Graph* from OSM will take some time, but the next times will be **inmediate** because the process is cached.

In [ ]:
def graph_Helsinki(place_name, network_type='drive'):
    '''
    Packed function for repetitive running and testing
    
    - place_name: string, area from OSM 
    - network_type: string, type of OSM network from drive, bike, walk
    
        return <Function>
    '''
    
    return ox.graph_from_place(place_name, network_type)

Cache the *Graph* fetch

In [ ]:
# cache
memory = Memory('cache', verbose=0)
graph_Helsinki_cached = memory.cache(graph_Helsinki)

In [ ]:
%%time

# get the road network as graph object
graph = graph_Helsinki_cached(place_name, network_type='drive')

The OSMNX function can plot the graph using the `plot_graph()` function. We added some modifications to have a clean view.

In [ ]:
fig, ax = ox.plot_graph(graph, node_size=0, edge_linewidth=0.2, edge_color='yellow', bgcolor='black')

### **6.5 Rautatieasema geometry as a Destination**

We are going to fetch the location of *Rautatieasema** from OSM stations. What we need is to get the *geometry* object and we will do it as simple as possible.

Simply, we create a function that fetch the location and return the *geometry* directly.

In [ ]:
def get_Rauratientori_gdf():
    '''
    Give back Rautatori Point geometry in WGS84 from OSM
        Return Point geometry 
    '''
    
    # Fetch train stations
    place_name = 'Capital region, Mainland Finland, Finland'

    # Tags for train station
    tags = {'railway': 'station'}

    # run
    all_stations = ox.features_from_place(place_name, tags)
    
    # reset index and remove subways
    all_stations = all_stations.reset_index(drop=False)
    all_stations = all_stations[['name', 'geometry']]
    
    # get Rautatieasema
    Rautatientori = all_stations.loc[all_stations.name=='Helsinki']
    
    return (Rautatientori.geometry[0])

In [ ]:
# Rautatientori geom as destination

destin_geom = get_Rauratientori_gdf()

### **6.7 Calculate Shortest Path using Parallelization (16 Core)**

We are going to run step by step the process of finding the *Shortest Path*. Also, you will see that some steps are parallelized in the available cores.

The advantage we have separating the workflow is that we are able to run in parallel and it saves time processing

#### **6.7.1 About the Core Parallelization**

To start understanding a bit more the parallel computing let's take a look the available resources in our machine using the next command.


In [ ]:
!lscpu | egrep 'Model name|Socket|Thread|NUMA|CPU\(s\)'

We can see the number of Cores available in the **CPU(s)** sections. The partition contains in total 40 Cores but the configuration we specified at the beginning in the machine was 16 Cores. So? Despite 40 cores are available only 16 cores are *reserved* for our analysis.

We can monitor our processes and the Core utilization with `htop` library that let us visuallize dinamically how the computer is using the resources. If you are willing to monitor your process you should open in a new command line `$ htop` (be sure you are using your own environment)

Let's take a look at the resources utilization after each process.

#### **6.7.2 Find the Closest Nodes**

The *Shortest Path* is operated in the OSM network between nodes (intersections) so the Origin and Destination location are simply a reference that helps to find the closest node where the routing should start.

In the next cells we are going to find the closest nodes using the function `nearest_nodes()` from OSMNX.

The destination and only one geometry of Rautatieasema we computeed only 1 nearest node. Additiionally, the process of finding the closest note of every origin was parellized using the library `multiprocess`.

We are going to start measuring the processing time using simple time variables at the beginning and end of the cell and we will sum up at the end to compare how the efficiency improves using parallel computation.

##### **A1. Destination node**


In [ ]:
s = time.time()

# ---- A1) Closest destination node

closest_target_node = ox.nearest_nodes(G=graph, 
                                       X=destin_geom.x, 
                                       Y=destin_geom.y)


# --------------------------------------------------------
d1 = time.time() - s
print(f' - Closest destination node: {d1} seconds')

If you can monitor the processes using `htop` (optional) you will see that we have in total 40 Cores. Some of them are already in use but there are still 16 Cores available for our use. 

This is how the resource utilization looks if you are running the *Destination node* cell

![htop1](img/machine-core-1.png)

As we are running a common process the computer is using a single node: the Core #30. 

##### **A2. Origins nodes**

We have 19 070 origins that we should compute and find the closest node. To optimize the time processing we are going to parallelize for every origin.

We will use the Python library `multiprocess` to parallelize the closest node process of the origins. Let's start packing the function `nearest_nodes()` from OSMNX.

In [ ]:
print(f'- Processing total routes: {len(all_origins)}')

In [ ]:
def get_nearest_node(graph, geom):
    '''
    Packed function to return a closest node from geometry
    
    - graph: graph, osm object
    - origin_geom: point geom, individual
    
        return:: int, a node code
    '''
    
    # calculate node
    node = ox.nearest_nodes(G=graph, X=geom.x, Y=geom.y) 
    
    return node

The `multiprocess` library is able to send a job to every core and optimize the workflows. The step is simple once we have packed the function we are going to use in this case `get_nearest_node()`

##### *Note*
As a condition, you should be able to loop over the inputs otherwise the process is not going to be parallelized. We are looping over every origin so we can distribute the process in every core.

The variables involved in the parallelization are:

- `cpus`: the number of cores available for computation with `cpu_count()`
- `args`: the parameters of our packed function, looping over every input
- `pool`: a pool opened for every available core where that *asyncronize* the process with `starmap_async()`
- `get()`: catch the result of every process in every core and save in a list

Then, you to close the pool you `close()` and `join()`.

In [ ]:
s = time.time()

# ----- A2) Closest origins node

# get cores
cpus = mp.cpu_count()

# ----- args

args = ((graph, geom) for geom in all_origins)

# ----- pool - with function

pool = mp.Pool(cpus)
sma = pool.starmap_async(get_nearest_node, args)

# ----- get list of results

closest_origin_node_list = sma.get()
pool.close()
pool.join()


# ----------------------------------------------------------
d2_parallel = time.time() - s
print(f' - Closest node of origins [PARALLELIZED]: {d2_parallel/60} minutes')

If you monitor the process in `htop` you will notice how the resource utilization changed.

At first you will see how the Cores start to be used:

![htop2](img/machine-core-starting.png)

Finally, it will occupy the 16 Cores available: 

![htop1](img/machine-core-16.png)

In total 16 Cores: from Core #20 to Core #35 

#### **6.7.3 Calculating the Shortest Path**

The OSMNX library has implemented the Shortest Path algoritm using the function `shortest_path()`. A particularity of this function is that it create routes based on OD paired-list. For example, if your input is `Origins=[O1, O2, O3]` and `Destinations=[D1, D2, D3]`, the output paths are going to be defined as `[O1 to D1, O2 to D2, O3 to D3]`.

Based on this, we need to have two lists with the same length. As we are using only 1 destination, we will multiply by the length of the origins so we have all routes to the same destination.

Fortunately, the OSMNX library has implemented in the function a parallelization parameter `cpus`. If you want to use all available cores, in this case 16 cores, you need to add `cpus=None` and it will use all available resources.

Follow the next code.

In [ ]:
s = time.time()

# ---- target nodes should be same length as origins

closest_target_node_list = [closest_target_node] * len(closest_origin_node_list)

# --------6.7.3) Shortest Path ---------

# run
routes = ox.shortest_path(graph, 
                         orig = closest_origin_node_list, 
                         dest = closest_target_node_list, 
                         weight = 'length',
                         cpus = None) # none takes all cores, default=1


# --------------------------------------------------------------------------

d3_parallel = time.time() - s
print(f' - Shortest Path run [PARALLELIZED]: {d3_parallel/60} minutes')

As we explained, the routes are defined between nodes, and might the case that some nodes are too close there are no existing routes and in those cases the OSMNX return a `None` object.

In this line we are going to remove the invalid routes.

In [ ]:
# remove None values from invalid routes
all_routes = [value for value in routes if value != None]

# check routes length
len(all_routes)

#### **6.7.4 From nodes to paths**

The output of the Shortest Path is a list of node's codes that construct the route we want. Then, we need to extract the line geometries using those nodes.

Be aware that we can do this work using the already implemented function in OSMNX `utils_graph.route_to_gdf()`. Particularly, it will give a row for each route segment and it will create quite many routes for the ~19 000 routes we are working on. So, we will create a function `nodes_to_path()` that will transform all nodes to to a single line geometry.

Get familiar with the function.

In [ ]:

def nodes_to_path(graph, route_nodes):
    '''
    Function to transform node's path to a single line geometry
    - graph: osm graph object
    - route_nodes: list, nodes of path
        return geodataframe with a single row
    '''
    
    # get route from nodes - get only geom and length
    shortest_route_parallel = ox.utils_graph.route_to_gdf(graph, route_nodes, weight='length')[['length', 'geometry']].reset_index(drop=True)
    
    # join the road segments
    multi_line = MultiLineString([linegeom for linegeom in shortest_route_parallel.geometry])    
    total_length = sum([value for value in shortest_route_parallel['length']])
    
    merged_line = ops.linemerge(multi_line)
    
    # new gdf
    shortest_route_merged = gpd.GeoDataFrame(columns=['geometry'], geometry='geometry', crs=4326)
    shortest_route_merged.at[0, 'geometry'] = merged_line
    shortest_route_merged.at[0, 'length'] = total_length
        
    return shortest_route_merged

Then, we will use again a `pool` from `multiprocess` to parallelize this path geometry creation. You can monitor also the core usage using the `htop`

In [ ]:
s = time.time()

# --------- 6.7.4) Get path as geom GDF

# get cores
cpus = mp.cpu_count()

# ----- args

args = ((graph, route_nodes) for route_nodes in all_routes)

# ----- pool - with function

pool = mp.Pool(cpus)
sma = pool.starmap_async(nodes_to_path, args)

# ----- get list of results - GDF

shortest_path_gdf_list = sma.get()
pool.close()
pool.join()

# --------------------------------------------------------------------------

d4_parallel = time.time() - s
print(f' - Nodes to path: {d4_parallel/60} minutes')

The output will be a list of Shortest Paths stored in the variable `shortest_path_gdf_list`. We will gather all results into one geodataframe using `concat()` function.

In [ ]:
# ------  create a single GDF

all_routes_gdf = pd.concat(shortest_path_gdf_list)  

We have got the length of every route fetching the same column from OSM that probably was calculated at global level. We would like to calculate a more accurate distance computed in a Finnish CRS (3387) so we need to operate a new column and add the newly measured distance.

We will create a function.

In [ ]:
def compute_fin_distance(shortest_path_gdf):
    '''
    Compute distance in FinCRS EPSG:3387
    
    - shortest_path_gdf: geodataframe, with geometry colum
        return geodataframe, with additional distance column
    
    '''
    
    # project WGS84 to EPSG3387
    distances = shortest_path_gdf.to_crs("EPSG:3387").geometry.length
    
    # add
    shortest_path_gdf['distance_fincrs'] = distances
    
    return shortest_path_gdf

In [ ]:
# ----- Compute distance EPSG FIN

route_distance_gdf = compute_fin_distance(all_routes_gdf)

### **6.7.5 Results**

We are going to check how our final geodataframe looks like and count how many routes we have calculated in total.

We will see that there is a slightly difference with distances calculated in the Finnish CRS. If you are going to use this process to calculate statistics it is recommended to do it using the distances from the local CRS.

In [ ]:
print(f' - In total {len(all_routes_gdf)} routes processed using 16 cores\n')

route_distance_gdf.head()

A quick plot to visualize the routes.

In [ ]:
ax = all_routes_gdf.plot(figsize=(10, 8), 
                         column='distance_fincrs', 
                         linewidth=0.5, 
                         cmap='inferno', 
                         alpha=1
                        );

ax.axis(False);

plt.savefig('output/shortest_path_parallel_16_core.png', dpi=300)

Finally, we are going to save in our local disk the routes as GIS format. Geopackage in this case.

In [ ]:
s = time.time()

# -------- Save

filepath = 'output/shortest_path_parallel_16_core.gpkg'
all_routes_gdf.to_file(filepath, index=False)

# --------------------------------------------------------------------------

dsave1 = time.time() - s
print(f' - Saving time: {dsave1} seconds')

### **6.8 Calculate Shortest Path with 1 Core**

#### **6.8.1 Do I have to configure 1 Core?**

No, the process by default is run in a single Core, as it was run in a personal laptop. But, in this section there are not modification in the code so parallelization is not active.

#### **6.8.2 Find the Closest Nodes**

As we have already calculated the closest node of the destination we will run only the closest node calculation for the origins, but this time using a single core.

##### **A2. Origins nodes**

In [ ]:
s = time.time()

# ----- A2-non) Closest origins node

closest_origin_node_list = [ox.nearest_nodes(G=graph, 
                                           X=origin_geom.x, 
                                           Y=origin_geom.y) 
                               for origin_geom in all_origins]
    
# ----------------------------------------------------------

d2_non = time.time() - s

print(f' - Closest node of origins [NON-PARALLELIZED]: {d2_non/60} minutes')

#### **6.8.3 Calculating the Shortest Path (1 Core)**

Then we will run the shortest path using a single core. 

In [ ]:
s = time.time()

# ---- target nodes should be same length as origins

closest_target_node_list = [closest_target_node] * len(closest_origin_node_list)

# --------6.8.3) Shortest Path ---------

# run
routes = ox.shortest_path(graph, 
                         orig = closest_origin_node_list, 
                         dest = closest_target_node_list, 
                         weight = 'length',
                         cpus = 1)


# --------------------------------------------------------------------------

d3_non = time.time() - s
print(f' - Shortest Path run [NON-PARALLELIZED]: {d3_non/60} minutes')

We will remove non valid routes and check how many are valid.

In [ ]:
# remove None values from invalid routes
all_routes = [value for value in routes if value != None]

# check routes length
len(all_routes)

#### **6.8.4 Nodes to path**

The nodes to path are computed in this part with a single core.

Beforehand, the results were gathered in a list using `pool` from parallelization. But now, we are going to loop over each path (as nodes) and use the function previously created that return a single line geometry. On the fly, we will calculate the distance using the Finnish CRS.

Finally, we will have a single geodataframe with all routes.

In [ ]:
s = time.time()

# --------- 7.4) Get path as geom GDF

all_routes_gdf = gpd.GeoDataFrame()

for each_route in routes:
        
    # ----- create nodes path to GDF
    
    route_gdf = nodes_to_path(graph, each_route)
    
    # ----- Compute distance

    route_distance_gdf = compute_fin_distance(route_gdf)
    
    all_routes_gdf = pd.concat([all_routes_gdf, route_distance_gdf])
    
# --------------------------------------------------------------------------

d4_non = time.time() - s
print(f' - Nodes to path and distances: {d4_non/60} minutes')

#### **6.8.5 Results (1 Core)**

Let's take a look at the results.

In [ ]:
print(f' - In total {len(all_routes_gdf)} routes processed with 1 core\n')

all_routes_gdf.head()

Let's do a quick map

In [ ]:
ax = all_routes_gdf.plot(figsize=(10, 8), 
                         column='distance_fincrs', 
                         linewidth=0.5, 
                         cmap='turbo', 
                         alpha=1
                           );

ax.axis(False);

plt.savefig('output/shortest_path_parallel_1_core.png', dpi=300)

We will save the results.

In [ ]:
s = time.time()

# -------- Save

filepath = 'output/shortest_path_parallel_1_core.gpkg'
all_routes_gdf.to_file(filepath, index=False)

# --------------------------------------------------------------------------

dsave2 = time.time() - s
print(f' - Saving time: {dsave2} seconds')

## **7. SUMMARY**

We are going to print the sum of all processing times and compare how benefitial was to use the parallelization. Be aware that:

- `d1` corresponds to the closest node of destination ()
- `d2` corresponds to the closest node of origins (buildings)
- `d3` corresponds to the shortest path calculation
- `d4` corresponds to the transformation from nodes to line geometry

Then, saving times are separated. Both should be the same time.

In [ ]:
print(f'--------------------- PROCESSING TIME --------------------\n')

total_parallel = d1 + d2_parallel + d3_parallel + d4_parallel

print(f'----- Total parallelization time : {round(total_parallel/60, 2)} mins -> 16 Core')

total_non = d1 + d2_non + d3_non + d4_non

print(f'----- Total time : {round(total_non/60, 2)} mins -> 1 Core')

print(f'--------------------- SAVING TIME --------------------\n')

print(f'-- Saving time parallel result: {round(dsave1, 2)} seconds')
print(f'-- Saving time 1 core result: {round(dsave2, 2)} seconds')